In [1]:
import io
from pytrends.request import TrendReq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import webbrowser
import os
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from PIL import Image
import datetime
from dateutil.relativedelta import relativedelta
import random

# Global Values

In [2]:
#current_datetime = datetime.datetime.now().strftime("%Y-%m-%d")
current_datetime = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
random_min=50.45
random_max=86.79

# Inputs

In [3]:
#Storing data from file Panda DF
input_keywords_df = pd.read_excel('Keywords_List.xlsx')

#Storing keywords to a list
input_keyword_list = input_keywords_df["Keywords"].values.tolist()
#Removing any repeating keywords
input_keyword_list = list(set(input_keyword_list))

# Storing Location and Timeframe data
geo_input = input_keywords_df["Location"].values.tolist()
geo_input=geo_input[0]
timeframe_input = input_keywords_df["Timeframe"].values.tolist()
timeframe_input=timeframe_input[0]


####################################################################
print("Location             :",geo_input)

#Time Frmae Data
timeframe_list=timeframe_input.split()
end_date=timeframe_list[0]
start_date=timeframe_list[1]
print("Start Date           :",start_date)
print("End Date             :",end_date)

# Convert the start date string to a datetime.date object
start_date_2 = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
# Calculate the date one year before the start date
previous_year_date = start_date_2 - relativedelta(years=1)
print("Previous Year Date   :",previous_year_date)

print("Keywords>>")
print(input_keyword_list)

Location             : US
Start Date           : 2023-09-15
End Date             : 2018-09-15
Previous Year Date   : 2022-09-15
Keywords>>
['American padlock', 'Assa Abloy padlock', 'Medeco padlock', 'Squire padlock', 'Abloy padlock', 'ABUS padlock', 'Yale padlock', 'Schlage padlock', 'Kwikset padlock']


# Reference Keyword | Among 5 Random sample

In [4]:
#Choosing a good candidate keyword to keep as reference for rest of the keywords by running a trial Google Trends
#Taking out first 5 keywords as sample
trial_keywords=input_keyword_list[:5]

#Trial Pytrends - Google Trends connection request for trial keywords
trial_pytrends_request=TrendReq()

# Generate a random delay between, for example, 10 and 20 seconds
delay_seconds = random.uniform(random_min, random_max)
print("sleep_start",delay_seconds)
time.sleep(delay_seconds)  

#Google Trends data loaded based on Location and Timeframe for trial keywords
trial_pytrends_request.build_payload(trial_keywords,geo=geo_input,timeframe=timeframe_input)

# Generate a random delay between, for example, 10 and 20 seconds
delay_seconds = random.uniform(random_min, random_max)
print("sleep_middle",delay_seconds)
time.sleep(delay_seconds)  

#Interest over time is stored for trial keywords - output is Pandas data frames
trial_pytrends_df=trial_pytrends_request.interest_over_time()

#Dropping "isPartial" column from the pytrends dataframe
trial_pytrends_df.drop(columns=["isPartial"], inplace=True)

# Average values of each keywords and keywords list
trial_average=trial_pytrends_df.mean().tolist()
trial_keywords=trial_pytrends_df.columns.tolist()

#Sorting/Ranking the keywords
np_trial_average=np.array(trial_average)
np_trial_keywords=np.array(trial_keywords)

np_trial_average_sorted_index=np.argsort(np_trial_average)
trial_average_sorted=np_trial_average[np_trial_average_sorted_index].tolist()
trial_keywords_sorted=np_trial_keywords[np_trial_average_sorted_index].tolist()

#Storing First Keyword
#hich will be used as linking keyword across all sets and will be used to normalise with resprct to each other later
first_keyword=trial_keywords_sorted[-1]

print("Completed")

sleep_start 68.51260202232588
sleep_middle 67.40378655650323
Completed


In [5]:
print(trial_keywords_sorted)
print(trial_average_sorted)
print("First/Reference Keywrd:",first_keyword)

['Assa Abloy padlock', 'Medeco padlock', 'Squire padlock', 'Abloy padlock', 'American padlock']
[0.0, 5.919540229885057, 6.363984674329502, 10.544061302681992, 17.35249042145594]
First/Reference Keywrd: American padlock


#Harcoding First Keyword
#first_keyword="Keyed alike padlock"

# 4 Set and 5 Set Keyword Preparation

In [6]:
#Removing first keyword from main set and storing rest of the keyword
rest_of_keywords=input_keyword_list[:]
rest_of_keywords.pop(rest_of_keywords.index(first_keyword))

#List to store sets/lists of 5 keywords
keyword_sets=[]

#Making sets/lists of 4 keywords from rest all keywords
for i in range(0,len(rest_of_keywords),4):
    keyword_sets.append(rest_of_keywords[i:i+4])
#Now inserts first/reference keyword in all the above sets/lists which has only 4 keywords
for i in range(0,len(keyword_sets)):
    keyword_sets[i].insert(0,first_keyword)

In [7]:
print(f'{len(keyword_sets)} Total Sets of Keywords with "{first_keyword}" as the Reference Keyword')
print(keyword_sets)

2 Total Sets of Keywords with "American padlock" as the Reference Keyword
[['American padlock', 'Assa Abloy padlock', 'Medeco padlock', 'Squire padlock', 'Abloy padlock'], ['American padlock', 'ABUS padlock', 'Yale padlock', 'Schlage padlock', 'Kwikset padlock']]


# Pytrends API > Google Trends Conenction

In [8]:
#List created to store Google Trends Connection request for each set of 5 keywords
pytrends_request_list=[]
pytrends_df_list=[]

for i in range(len(keyword_sets)):
    #Google Trends Connection requests stored
    pytrends_request_list.append(TrendReq())
    
    # Generate a random delay between, for example, 10 and 20 seconds
    delay_seconds = random.uniform(random_min, random_max)
    print("sleep_start",i,delay_seconds)
    time.sleep(delay_seconds)  
    
    #Keyword Data points are requested based on Location and Timeframe
    pytrends_request_list[i].build_payload(keyword_sets[i],geo=geo_input,timeframe=timeframe_input)
    
    # Generate a random delay between, for example, 10 and 20 seconds
    delay_seconds = random.uniform(random_min, random_max)
    print("sleep_middle",i,delay_seconds)
    time.sleep(delay_seconds)
    
    #Interest over time is stored for each set/list - output is Pandas data frames, which is stored in a list for each set
    pytrends_df_list.append(pytrends_request_list[i].interest_over_time())
    
    # Generate a random delay between, for example, 10 and 20 seconds
    delay_seconds = random.uniform(random_min, random_max)
    print("sleep_end",i,delay_seconds)
    time.sleep(delay_seconds)  
    
    #Dropping "isPartial" column from the pytrends dataframe
    pytrends_df_list[i].drop(columns=["isPartial"], inplace=True)
    
        
gta_df_backup=pytrends_df_list.copy()
print("Completed Google")

sleep_start 0 67.5196560198105
sleep_middle 0 83.8103983701472
sleep_end 0 65.2586220482471
sleep_start 1 59.29130481524049
sleep_middle 1 80.77323194894583
sleep_end 1 50.8276593579569
Completed Google


#Refresh Dataframe
#pytrends_df_list=gta_df_backup.copy()

# DATA WRANGLING

# Combining & Normalization of all DF

In [9]:
#Refresh Dataframe
pytrends_df_list=gta_df_backup.copy()

#### Try taking rank only for latt year | Clean the datframe index date and filter for last year

# Max of the reference keyword is is exttracted from all dataframes
maxReferenceKeyword=[]
for i in range(len(pytrends_df_list)):
    maxReferenceKeyword.append(pytrends_df_list[i][first_keyword].max())
    
#Normalization factor is obtained uaing reference keyword
normalizationFactor=[]
for i in range(len(pytrends_df_list)):
    normalizationFactor.append(maxReferenceKeyword[0]/maxReferenceKeyword[i])

#All the Dataframes are Normalized using Normalization Factor
for i in range(len(pytrends_df_list)):
    pytrends_df_list[i]=pytrends_df_list[i]*normalizationFactor[i]

#Removing - the first elemnt data/column from Dataframe in the first set/list which was repeated in rest of the set/list
for i in range(1,len(pytrends_df_list)):
    pytrends_df_list[i].drop(columns=[first_keyword],inplace=True)

#Combined Dataframe
combined_df=pytrends_df_list[0]
for i in range(1,len(pytrends_df_list)):
    combined_df=combined_df.join(pytrends_df_list[i])

#Extracting Max value from the dataframe, and use it to normalise the entire dataframe so that max is 100
max_trend_datapoint=combined_df.to_numpy().max()

# Normalising the combined dataframe to 100 scale
normalise100_factor=(100-max_trend_datapoint)/max_trend_datapoint
normalise100_factor=1+normalise100_factor
normalised_df=combined_df*normalise100_factor
normalised_df=normalised_df.round(2)

# Ranking the Keywords | Previous Year Performance

In [10]:
#Ranking to be been calculated based on previous year popularity
normalised_df_1year=normalised_df[previous_year_date:start_date]

# Mean/Average values of each keywords and keywords list
average_list=normalised_df_1year.mean().tolist()
keyword_list=normalised_df_1year.columns.tolist()

#List to Numpy
np_average_list=np.array(average_list)
np_keyword_list=np.array(keyword_list)

# Sorting/Ranking the average
np_sorted_avg_index=np.argsort(np_average_list)

np_sorted_average=np_average_list[np_sorted_avg_index][::-1]

# Normalizing the average to 100
normalise100factor_np=(100-np_sorted_average[0])/np_sorted_average[0]
normalise100factor_np=1+normalise100factor_np
normalised_avg_list_np=np.round_(np_sorted_average*normalise100factor_np,1)

# Sorting/Ranking the keywords
np_sorted_keywords=np_keyword_list[np_sorted_avg_index][::-1]

# Sorting the Combined Multi-Year DataFrame based on last year performance

In [11]:
sorted_df=normalised_df[np_sorted_keywords]

print(normalised_avg_list_np)
print(np_sorted_keywords)
print(sorted_df)

# URLs Generation

In [12]:
keywords_string_list = []
joined_keywords = ""
counter = 0

for i in range(len(np_sorted_keywords)):
    joined_keywords += "," + np_sorted_keywords[i]  # Join the current element with a comma
    counter += 1

    if counter == 5 or i == len(np_sorted_keywords) - 1:
        keywords_string_list.append(joined_keywords[1:])  # Append the joined string, excluding the leading comma
        counter = 0
        joined_keywords = ""
        
        joined_keywords += "," + np_sorted_keywords[i]  # Join the current element with a comma
        counter += 1

urls=[]
url_base_string="https://trends.google.com/trends/explore?date="+end_date+"%20"+start_date+"&geo="+geo_input+"&q="

for i in keywords_string_list:
    url=url_base_string+i
    urls.append(url)

urls

['https://trends.google.com/trends/explore?date=2018-09-15%202023-09-15&geo=US&q=ABUS padlock,American padlock,Yale padlock,Abloy padlock,Schlage padlock',
 'https://trends.google.com/trends/explore?date=2018-09-15%202023-09-15&geo=US&q=Schlage padlock,Kwikset padlock,Squire padlock,Medeco padlock,Assa Abloy padlock']

# Custom Output

#Top5List
normalised_avg_list5_np=normalised_avg_list_np[:5]
np_sorted_keywords5=np_sorted_keywords[:5]
top_5df=normalised_df[np_sorted_keywords5]

#Top10List
normalised_avg_list10_np=normalised_avg_list_np[:10]
np_sorted_keywords10=np_sorted_keywords[:10]
top_10df=normalised_df[np_sorted_keywords10]

#Top15List
normalised_avg_list15_np=normalised_avg_list_np[:15]
np_sorted_keywords15=np_sorted_keywords[:15]
top_15df=normalised_df[np_sorted_keywords15]

# Plotting 15

y_pos=np.arange(len(np_sorted_keywords15))
plt.barh(y_pos,normalised_avg_list15_np[::-1],align='center',alpha=0.5)
plt.yticks(y_pos,np_sorted_keywords15[::-1])
plt.xlabel('Average ')
plt.show()

# Excel Reporting/Downloads

In [13]:
#Creating Rank numbers to populate in Excel
ranks=np.arange(1,len(np_sorted_keywords)+1)

#Adding Keywords and their Averages
ranking_np= np.array([ranks,np_sorted_keywords,normalised_avg_list_np])

#Trasposing the NDArray
ranking_np=ranking_np.transpose()

#Numpy to Pandas
ranking_df=pd.DataFrame(data=ranking_np, columns=["Rank","Keywords", "Average-1Year"])

# Create a folder for Links with the timestamp
folder_name = f'GTA_Ranking_{current_datetime}'
os.makedirs(folder_name, exist_ok=True)

# Define the Excel file names with the timestamp
excel_file_name = f'Keyword_Ranking_{current_datetime}.xlsx'

#Save the DataFrame to an Excel file in the folder
excel_file_path = os.path.join(folder_name, excel_file_name)
ranking_df.to_excel(excel_file_path,index=False)

# Print a message to confirm the completion
print(f"Excel file '{excel_file_name}' saved in folder '{folder_name}'")

Excel file 'Keyword_Ranking_2023-09-29_18-40-21.xlsx' saved in folder 'GTA_Ranking_2023-09-29_18-40-21'


# Create a folder for Links with the timestamp
folder_name = f'GTA_Datapoints_{current_datetime}'
os.makedirs(folder_name, exist_ok=True)

# Define the Excel file names with the timestamp
excel_file_name = f'Trends_Datapoints_{current_datetime}.xlsx'

#Save the DataFrame to an Excel file in the folder
excel_file_path = os.path.join(folder_name, excel_file_name)
sorted_df.to_excel(excel_file_path,index=True)

# Print a message to confirm the completion
print(f"Excel file '{excel_file_name}' saved in folder '{folder_name}'")

# Download URLs

In [14]:
# Create a folder for Links with the timestamp
folder_name = f'GTA_Links_{current_datetime}'
os.makedirs(folder_name, exist_ok=True)

# Create a DataFrame with your URLs
links_df = pd.DataFrame(urls)

# Define the Excel file name with the timestamp
excel_file_name = f'Trend_Links_{current_datetime}.xlsx'

# Save the DataFrame to an Excel file in the folder
excel_file_path = os.path.join(folder_name, excel_file_name)
links_df.to_excel(excel_file_path, index=False)

# Print a message to confirm the completion
print(f"Excel file '{excel_file_name}' saved in folder '{folder_name}'")

Excel file 'Trend_Links_2023-09-29_18-40-21.xlsx' saved in folder 'GTA_Links_2023-09-29_18-40-21'


# Auto Link Open in Browser Feature
Works

# Set the time delay (in seconds) between opening links
time_delay = 2  # Adjust this value as needed
for url in urls:
    webbrowser.open(url)
    time.sleep(time_delay)

print("Completed")

# Screen Shot Feature
Works, but Not accurate

# Create a folder with a timestamp to store the screenshots
screenshot_folder = f'GTA_Screenshots_{current_datetime}'
os.makedirs(screenshot_folder, exist_ok=True)

# Initialize the Edge browser driver with headless mode
edge_options = webdriver.EdgeOptions()
edge_options.add_argument('--headless')  # Enable headless mode
driver = webdriver.Edge(options=edge_options)

# Initialize a counter for image names
image_counter = 1

# Set the time delay (in seconds) between capturing screenshots
time_delay = 5  # Adjust this value as needed
time.sleep(time_delay)

for url, keyword_string in zip(urls, keywords_string_list):
    # Navigate to the web page
    driver.get(url)

    # Wait for the page to load (you can adjust the sleep time as needed)
    time.sleep(5)

    # Capture a full-page screenshot of the web page
    screenshot_filename = os.path.join(screenshot_folder, f'{image_counter:03d}_{keyword_string}.png')
    driver.save_screenshot(screenshot_filename)

    # Open the screenshot using Pillow (PIL)
    img = Image.open(screenshot_filename)
    
    ## Define the cropping coordinates (left, top, right, bottom)
    ## Adjust these values to crop the image as desired
    #left = 50
    #top = 80
    #right = 900
    #bottom = 900

    ## Crop the image
    #img = img.crop((left, top, right, bottom))

    # Save the full-page screenshot
    img.save(screenshot_filename)

    # Increment the image counter
    image_counter += 1
    
    # Add a time delay before capturing the next screenshot
    time.sleep(time_delay)

# Close the browser
driver.quit()
print("Completed")